<a href="https://colab.research.google.com/github/GenAIUnplugged/langchain_series/blob/main/Gradio_langchain_basic_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain-core langchain-community langchain_openai pyMuPDF gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 

In [2]:
import gradio as gr
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [3]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [4]:
llm = ChatOpenAI(temperature=0,model="gpt-4o-mini")

In [5]:
memory = ConversationBufferMemory()

<ipython-input-5-a33ea3a4cc0d>:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


the below code connects the llm and the memory together \
Every time you call conversation.predict(input=...), it uses the entire conversation stored in memory to provide a contextual reply.

In [6]:
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=memory
)

<ipython-input-6-f98f30686eec>:1: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(


This is the main chatbot logic that runs on every message:

* Takes user_input from the text box.
* Calls conversation.predict(...) to get the model’s reply using full memory.
* Appends the input-output pair to the history list (which Gradio uses to render the chat visually).
* Returns the updated history twice — once for display in the UI and once for internal state management.

The history variable here is only used by Gradio, not by ConversationBufferMemory.\
history is for Gradio UI:
**gr.Chatbot() expects a list of (user_input, bot_response) tuples to display the conversation.** \
So history.append((user_input, response)) updates the visible chat window. \
**Format is List of tuples like [(user, bot), (user2, bot2)]** \\

**ConversationBufferMemory is internal to LangChain:**
* It does not use the history variable from Gradio.
* Instead, it automatically tracks the full conversation context every time you call response = conversation.predict(input=user_input)
* It stores that internally and appends new messages behind the scenes.
* When you use ConversationBufferMemory, LangChain automatically formats the prompt sent to the LLM to include the full conversation history. \

**Under the Hood**
When we call response = conversation.predict(input="What's the capital of Italy?")
1. Retrieves prior messages from ConversationBufferMemory
2. Formats them like: \
Human: Hi \
AI: Hello! \
Human: What's my name? \
AI: You said it's Sarah. \
Human: What's the capital of Italy? \
3. Sends the whole thing as a prompt to the LLM.
4. Gets the response and updates memory.

If you don’t want this default behavior, you can: \
Use your own memory class Override how the prompt is built \
**Examples:**
* ConversationSummaryMemory: Summarizes instead of verbatim history.
* ChatMessageHistory: Keeps messages structured (like OpenAI chat format).

In [10]:
def chat(user_input, history):
    response = conversation.predict(input=user_input)
    history.append((user_input, response))
    return history, history

* gr.Blocks(): A flexible layout system for building interfaces.
* gr.Markdown(...): Adds a title at the top.
* gr.Chatbot(): The UI element that shows the conversation.
* gr.Textbox(): Where users type their messages.
* gr.State([]): Keeps the chat history across interactions (Gradio needs this to remember prior messages).
* **msg.submit(...): When the user hits enter:**
* It calls the chat() function.
* Passes the current message and history.
* Updates both the visible chatbot and the stored state.

In [11]:
# Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# 🧠 LangChain Chatbot with Memory")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Your Message")
    state = gr.State([])  # This will store the chat history
    msg.submit(chat, [msg, state], [chatbot, state])

<ipython-input-11-644c0944c6c2>:4: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


In [12]:
demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4da84bef06095bd25d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**Understanding the difference between Gradio State and LangChain’s ConversationBufferMemory is key to building memory-aware chatbots.**
* Gradio’s utility to store temporary values between interactions.
* Think of it as a global variable that survives UI updates.
* Not visible to LLM. Only backend data to show the conversation history for the front end

**ConversationBufferMemory (LangChain)**
* Keeps the raw input-output message history in a format the LLM can understand.
* Automatically inserts previous turns into the prompt passed to the LLM

**In conclusion:Use Both Together**
* gr.State → Keeps chat history for the UI (gr.Chatbot() needs it).
*ConversationBufferMemory → Keeps message history for the LLM so it responds contextually.